# Contextual Word Embeddings

Como hemos visto, algoritmos como word2vec, GloVe o fastText aprenden a calcular de manera no supervisada representaciones continuas de palabras o de segmentos de palabras. Sin embargo, estas representaciones son estáticas y no tienen en cuenta los distintos significados que una misma palabra pueda tener.

Y el lenguaje natural está repleto de ambigüedades, a varios niveles, y existen fenómenos como la **polisemia** y la **homografía**, que implican que una misma cadena de caracteres pueda tener más de un significado.

- *banco*: de sentar, de crédito, de peces, de arena
- *cubo*: de agua, operación matemática, figura geométrica
- *planta*: arbusto, fábrica, del pie
- *bajo*: nombre, adjetivo, preposición

## ELMo

![](img/elmo.gif)

ELMo (Peters et al., 2018) es una nueva técnica para calcular y asignar vectores de palabras de manera dinámica.

Los vectores que asigna ELMo a las palabras cambian dependiendo del contexto porque tiene en cuenta todo el texto.

El algoritmo combina dos modelos de lenguaje para calcular probabilidades de una palabra considerando su contexto anterior y posterior.